In [22]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from fuzzywuzzy import fuzz, process
import re
from collections import defaultdict
import warnings
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl

## Основной класс

In [39]:

class ColumnComparator:
    def __init__(self, similarity_threshold=80, data_similarity_threshold=0.7):
        self.similarity_threshold = similarity_threshold
        self.data_similarity_threshold = data_similarity_threshold
        self.column_profiles = {}
        
    def preprocess_column_name(self, name):
        """Предобработка названия колонки"""
        if pd.isna(name):
            return ""
        name = str(name).lower().strip()
        # Удаляем специальные символы и лишние пробелы
        name = re.sub(r'[^\w\s]', ' ', name)
        name = re.sub(r'\s+', ' ', name)
        return name.strip()
    
    def get_column_profile(self, df, column_name):
        """Создает профиль для колонки с метаданными и характеристиками"""
        series = df[column_name].dropna()
        
        profile = {
            'name': column_name,
            'preprocessed_name': self.preprocess_column_name(column_name),
            'dtype': str(df[column_name].dtype),
            'total_count': len(df),
            'non_null_count': len(series),
            'null_percentage': (df[column_name].isnull().sum() / len(df)) * 100,
            'unique_count': series.nunique(),
            'unique_percentage': (series.nunique() / len(series)) * 100 if len(series) > 0 else 0,
            'sample_values': series.head(5).tolist(),
            'value_lengths': series.astype(str).str.len().describe().to_dict() if len(series) > 0 else {}
        }
        
        # Определяем тип данных более точно
        if pd.api.types.is_numeric_dtype(series):
            profile['data_type'] = 'numeric'
            profile['stats'] = series.describe().to_dict()
        elif pd.api.types.is_datetime64_any_dtype(series):
            profile['data_type'] = 'datetime'
        else:
            profile['data_type'] = 'string'
            # Анализ паттернов для строковых данных
            if len(series) > 0:
                sample_str = series.astype(str).iloc[0]
                if re.match(r'^[A-Za-z]{2,3}-\d{4,6}$', sample_str):
                    profile['pattern'] = 'code_pattern'
                elif re.match(r'^\d{2}\.\d{2}\.\d{4}$', sample_str):
                    profile['pattern'] = 'date_pattern'
                elif '@' in sample_str:
                    profile['pattern'] = 'email_pattern'
        
        return profile
    
    def calculate_name_similarity(self, name1, name2):
        """Вычисляет схожесть названий колонок"""
        return fuzz.token_sort_ratio(name1, name2)
    
    def calculate_data_similarity(self, profile1, profile2):
        """Вычисляет схожесть данных в колонках"""
        similarity_score = 0
        factors_checked = 0
        
        # Сравнение типов данных
        if profile1['data_type'] == profile2['data_type']:
            similarity_score += 25
        factors_checked += 25
        
        # Сравнение уникальности
        uniqueness_diff = abs(profile1['unique_percentage'] - profile2['unique_percentage'])
        if uniqueness_diff < 20:  # Разница менее 20%
            similarity_score += 25
        factors_checked += 25
        
        # Сравнение распределения длин (для строковых данных)
        if (profile1['data_type'] == 'string' and profile2['data_type'] == 'string' and 
            profile1['value_lengths'] and profile2['value_lengths']):
            len_diff = abs(profile1['value_lengths'].get('mean', 0) - profile2['value_lengths'].get('mean', 0))
            if len_diff < 3:  # Средняя длина отличается менее чем на 3 символа
                similarity_score += 25
            factors_checked += 25
        
        # Сравнение паттернов
        if profile1.get('pattern') == profile2.get('pattern'):
            similarity_score += 25
            factors_checked += 25
        
        return (similarity_score / factors_checked) * 100 if factors_checked > 0 else 0
    
    def find_column_matches(self, df1, df2):
        """Находит соответствия между колонками двух DataFrame"""
        matches = []
        
        # Создаем профили для всех колонок
        profiles1 = {col: self.get_column_profile(df1, col) for col in df1.columns}
        profiles2 = {col: self.get_column_profile(df2, col) for col in df2.columns}
        
        for col1, profile1 in profiles1.items():
            best_match = None
            best_score = 0
            
            for col2, profile2 in profiles2.items():
                # Схожесть по имени
                name_similarity = self.calculate_name_similarity(
                    profile1['preprocessed_name'], 
                    profile2['preprocessed_name']
                )
                
                # Схожесть по данным
                data_similarity = self.calculate_data_similarity(profile1, profile2)
                
                # Комбинированная оценка (40% имя, 60% данные)
                combined_score = (name_similarity * 0.4) + (data_similarity * 0.6)
                
                if combined_score > best_score:
                    best_score = combined_score
                    best_match = (col2, name_similarity, data_similarity, combined_score)
            
            if best_match and best_score >= self.similarity_threshold:
                matches.append({
                    'system_a_column': col1,
                    'system_b_column': best_match[0],
                    'name_similarity': best_match[1],
                    'data_similarity': best_match[2],
                    'combined_score': best_match[3],
                    'status': 'MATCHED'
                })
            else:
                matches.append({
                    'system_a_column': col1,
                    'system_b_column': 'NO_MATCH',
                    'name_similarity': 0,
                    'data_similarity': 0,
                    'combined_score': 0,
                    'status': 'UNMATCHED'
                })
        
        return pd.DataFrame(matches), profiles1, profiles2
    
    def compare_matched_columns(self, df1, df2, matches_df, key_columns=None):
        """Сравнивает значения в сопоставленных колонках"""
        comparison_results = []
        matched_pairs = matches_df[matches_df['status'] == 'MATCHED']
        
        # Если указаны ключевые колонки для join, используем их
        if key_columns:
            key_col_a, key_col_b = key_columns
            merged_df = pd.merge(df1, df2, left_on=key_col_a, right_on=key_col_b, how='inner', suffixes=('_a', '_b'))
        else:
            # Простое построчное сравнение (осторожно - порядок может не совпадать)
            merged_df = pd.concat([df1.reset_index(drop=True), df2.reset_index(drop=True)], axis=1)
        
        for _, match in matched_pairs.iterrows():
            col_a = match['system_a_column']
            col_b = match['system_b_column']
            
            if col_a in df1.columns and col_b in df2.columns:
                if key_columns:
                    col_a_merged = col_a + '_a'
                    col_b_merged = col_b + '_b'
                else:
                    col_a_merged, col_b_merged = col_a, col_b
                
                if col_a_merged in merged_df.columns and col_b_merged in merged_df.columns:
                    # Сравниваем значения
                    values_a = merged_df[col_a_merged].fillna('')
                    values_b = merged_df[col_b_merged].fillna('')
                    
                    exact_matches = (values_a == values_b).sum()
                    total_comparisons = len(merged_df)
                    match_percentage = (exact_matches / total_comparisons) * 100 if total_comparisons > 0 else 0
                    
                    comparison_results.append({
                        'system_a_column': col_a,
                        'system_b_column': col_b,
                        'total_rows_compared': total_comparisons,
                        'exact_matches': exact_matches,
                        'match_percentage': match_percentage,
                        'mismatch_percentage': 100 - match_percentage
                    })
        
        return pd.DataFrame(comparison_results)

def analyze_folder_structure(base_path):
    """Анализирует структуру папок и находит пары файлов"""
    folder_pairs = []
    base_path = Path(base_path)
    
    for folder in base_path.iterdir():
        if folder.is_dir():
            excel_files = list(folder.glob('*.xlsx')) + list(folder.glob('*.xls'))
            if len(excel_files) == 2:
                folder_pairs.append({
                    'folder': folder.name,
                    'file1': excel_files[0],
                    'file2': excel_files[1]
                })
    
    return folder_pairs

In [44]:

class AdvancedColumnComparator(ColumnComparator):
    def __init__(self, similarity_threshold=75, data_similarity_threshold=0.7):
        super().__init__(similarity_threshold, data_similarity_threshold)
        self.mismatch_fill = PatternFill(start_color="FFE6E6", end_color="FFE6E6", fill_type="solid")
        self.header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
        self.summary_fill = PatternFill(start_color="DCE6F1", end_color="DCE6F1", fill_type="solid")
    
    def create_detailed_mismatch_report(self, df1, df2, matches_df, key_columns, folder_name, output_path):
        """Создает детальный отчет с расхождениями"""
        
        print("Начинаем создание отчета...")
        print(f"df1 колонки: {list(df1.columns)}")
        print(f"df2 колонки: {list(df2.columns)}")
        print(f"Key columns: {key_columns}")
        
        # Создаем Excel файл
        wb = Workbook()
        ws1 = wb.active
        ws1.title = "Детальные расхождения"
        
        # Получаем сопоставленные пары
        matched_pairs = matches_df[matches_df['status'] == 'MATCHED']
        print(f"Сопоставленные пары: {len(matched_pairs)}")
        
        # Создаем merge по ключевым колонкам
        key_col_a, key_col_b = key_columns
        merged_df = pd.merge(df1, df2, left_on=key_col_a, right_on=key_col_b, 
                           how='outer', suffixes=('_A', '_B'), indicator=True)
        
        print(f"После merge: {len(merged_df)} строк")
        print(f"Колонки после merge: {list(merged_df.columns)}")
        
        # Подготавливаем заголовки для отчета
        headers = [
            f'KEY_A_{key_col_a}',
            f'KEY_B_{key_col_b}', 
            'Status'
        ]
        
        # Добавляем колонки для каждой сопоставленной пары
        column_mapping = {}
        for _, match in matched_pairs.iterrows():
            col_a = match['system_a_column']
            col_b = match['system_b_column']
            headers.extend([f'A_{col_a}', f'B_{col_b}', f'Diff_{col_a}'])
            column_mapping[col_a] = col_b
        
        headers.append('Mismatch_Count')
        
        # Записываем заголовки
        for col_idx, header in enumerate(headers, 1):
            cell = ws1.cell(row=1, column=col_idx, value=header)
            cell.fill = self.header_fill
            cell.font = Font(bold=True, color="FFFFFF")
            cell.alignment = Alignment(horizontal='center')
        
        # Списки для статистики
        mismatch_rows_indices = []  # Индексы строк с расхождениями
        all_mismatches_count = 0    # Общее количество расхождений
        
        # ЗАРАНЕЕ ПОДСЧИТАЕМ РАСХОЖДЕНИЯ ПО КОЛОНКАМ
        column_mismatch_stats = []
        for col_a, col_b in column_mapping.items():
            col_mismatches = 0
            for _, row in merged_df.iterrows():
                col_a_merged = f'{col_a}_A'
                col_b_merged = f'{col_b}_B'
                
                val_a = row.get(col_a_merged, '')
                val_b = row.get(col_b_merged, '')
                
                # Если значения не найдены, пробуем найти в оригинальных колонках
                if val_a == '' and col_a in row:
                    val_a = row[col_a]
                if val_b == '' and col_b in row:
                    val_b = row[col_b]
                
                str_val_a = '' if pd.isna(val_a) else str(val_a)
                str_val_b = '' if pd.isna(val_b) else str(val_b)
                
                if str_val_a != str_val_b:
                    col_mismatches += 1
            
            column_mismatch_stats.append({
                'system_a_column': col_a,
                'system_b_column': col_b,
                'mismatch_count': col_mismatches,
                'total_comparisons': len(merged_df),
                'mismatch_percentage': (col_mismatches / len(merged_df) * 100) if len(merged_df) > 0 else 0
            })
        
        # Заполняем данные в лист
        row_idx = 2
        for idx, merged_row in merged_df.iterrows():
            row_data = {}
            mismatch_count = 0
            
            # Ключевые колонки
            key_a_val = merged_row.get(f'{key_col_a}_A', merged_row.get(key_col_a, ''))
            key_b_val = merged_row.get(f'{key_col_b}_B', merged_row.get(key_col_b, ''))
            
            row_data[f'KEY_A_{key_col_a}'] = '' if pd.isna(key_a_val) else key_a_val
            row_data[f'KEY_B_{key_col_b}'] = '' if pd.isna(key_b_val) else key_b_val
            row_data['Status'] = merged_row.get('_merge', 'both')
            
            # Данные по сопоставленным колонкам
            for col_a, col_b in column_mapping.items():
                col_a_merged = f'{col_a}_A'
                col_b_merged = f'{col_b}_B'
                
                # Получаем значения
                val_a = merged_row.get(col_a_merged, '')
                val_b = merged_row.get(col_b_merged, '')
                
                # Если значения не найдены, пробуем найти в оригинальных колонках
                if val_a == '' and col_a in merged_row:
                    val_a = merged_row[col_a]
                if val_b == '' and col_b in merged_row:
                    val_b = merged_row[col_b]
                
                # Преобразуем в строку и обрабатываем NaN
                str_val_a = '' if pd.isna(val_a) else str(val_a)
                str_val_b = '' if pd.isna(val_b) else str(val_b)
                
                # Сравниваем
                is_mismatch = str_val_a != str_val_b
                
                row_data[f'A_{col_a}'] = str_val_a
                row_data[f'B_{col_b}'] = str_val_b
                row_data[f'Diff_{col_a}'] = 'РАСХОЖДЕНИЕ' if is_mismatch else 'OK'
                
                if is_mismatch:
                    mismatch_count += 1
                    all_mismatches_count += 1
            
            row_data['Mismatch_Count'] = mismatch_count
            
            # Сохраняем информацию о строках с расхождениями
            if mismatch_count > 0:
                mismatch_rows_indices.append(idx)
            
            # Записываем строку в Excel
            for col_idx, header in enumerate(headers, 1):
                value = row_data.get(header, '')
                cell = ws1.cell(row=row_idx, column=col_idx, value=value)
                
                # Закрашиваем расхождения
                if 'РАСХОЖДЕНИЕ' in str(value):
                    cell.fill = self.mismatch_fill
                
                # Закрашиваем всю строку если есть расхождения
                elif header == 'Mismatch_Count' and value > 0:
                    for col in range(1, len(headers) + 1):
                        ws1.cell(row=row_idx, column=col).fill = self.mismatch_fill
            
            row_idx += 1
        
        # Настраиваем ширину колонок
        for column in ws1.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = min(max_length + 2, 30)
            ws1.column_dimensions[column_letter].width = adjusted_width
        
        # === ЛИСТ 2: Сводная статистика ===
        ws2 = wb.create_sheet("Сводная статистика")
        
        # Расчет статистики
        total_rows = len(merged_df)
        rows_with_mismatches = len(mismatch_rows_indices)
        rows_without_mismatches = total_rows - rows_with_mismatches
        
        total_columns_a = len(df1.columns)
        total_columns_b = len(df2.columns)
        matched_columns = len(matched_pairs)
        unmatched_columns_a = total_columns_a - matched_columns
        unmatched_columns_b = total_columns_b - matched_columns
        
        # Статистика по merge
        merge_stats = merged_df['_merge'].value_counts()
        left_only = merge_stats.get('left_only', 0)
        right_only = merge_stats.get('right_only', 0)
        both_sides = merge_stats.get('both', 0)
        
        # Расчет процента совпадения
        total_comparisons = total_rows * matched_columns
        
        if total_comparisons > 0:
            overall_match_percentage = 100 - (all_mismatches_count / total_comparisons * 100)
        else:
            overall_match_percentage = 0
        
        # ФИЛЬТРУЕМ КОЛОНКИ С РАСХОЖДЕНИЯМИ (после подсчета)
        columns_with_mismatches = [col for col in column_mismatch_stats if col['mismatch_count'] > 0]
        
        summary_data = [
            ["ПАРАМЕТР", "ЗНАЧЕНИЕ"],
            ["Общая информация", ""],
            ["Название папки", folder_name],
            ["Дата анализа", pd.Timestamp.now().strftime("%Y-%m-%d %H:%M")],
            ["", ""],
            ["Статистика по строкам", ""],
            ["Всего строк", total_rows],
            ["Строк без расхождений", rows_without_mismatches],
            ["Строк с расхождениями", rows_with_mismatches],
            ["Процент строк с расхождениями", f"{(rows_with_mismatches/total_rows)*100:.2f}%" if total_rows > 0 else "0%"],
            ["Строк только в системе A", left_only],
            ["Строк только в системе B", right_only],
            ["Строк в обеих системах", both_sides],
            ["", ""],
            ["Статистика по колонкам", ""],
            ["Всего колонок в системе A", total_columns_a],
            ["Всего колонок в системе B", total_columns_b],
            ["Сопоставленных колонок", matched_columns],
            ["Несопоставленных в системе A", unmatched_columns_a],
            ["Несопоставленных в системе B", unmatched_columns_b],
            ["Процент сопоставленных", f"{(matched_columns/min(total_columns_a, total_columns_b))*100:.2f}%" if min(total_columns_a, total_columns_b) > 0 else "0%"],
            ["Колонок с расхождениями", len(columns_with_mismatches)],
            ["", ""],
            ["Статистика по расхождениям", ""],
            ["Всего проверок значений", total_comparisons],
            ["Всего расхождений", all_mismatches_count],
            ["Общий процент совпадения", f"{overall_match_percentage:.2f}%"],
            ["Среднее расхождений на строку", f"{(all_mismatches_count/total_rows):.2f}" if total_rows > 0 else "0"]
        ]
        
        # Записываем сводную статистику
        for r_idx, row_data in enumerate(summary_data, 1):
            for c_idx, value in enumerate(row_data, 1):
                cell = ws2.cell(row=r_idx, column=c_idx, value=value)
                if r_idx == 1:
                    cell.fill = self.header_fill
                    cell.font = Font(bold=True, color="FFFFFF")
                    cell.alignment = Alignment(horizontal='center')
                elif any(x in str(value) for x in ["ПАРАМЕТР", "Общая информация", "Статистика по строкам", 
                                                 "Статистика по колонкам", "Статистика по расхождениям"]):
                    cell.fill = self.summary_fill
                    cell.font = Font(bold=True)
        
        # Статистика по колонкам с расхождениями
        if columns_with_mismatches:
            start_row = len(summary_data) + 2
            ws2.cell(row=start_row, column=1, value="Детали по колонкам с расхождениями").font = Font(bold=True)
            
            headers = ["Колонка A", "Колонка B", "Расхождения", "Всего проверок", "% Расхождений"]
            for c_idx, header in enumerate(headers, 1):
                cell = ws2.cell(row=start_row + 1, column=c_idx, value=header)
                cell.fill = self.header_fill
                cell.font = Font(bold=True, color="FFFFFF")
                cell.alignment = Alignment(horizontal='center')
            
            for m_idx, col_stats in enumerate(columns_with_mismatches, start_row + 2):
                ws2.cell(row=m_idx, column=1, value=col_stats['system_a_column'])
                ws2.cell(row=m_idx, column=2, value=col_stats['system_b_column'])
                ws2.cell(row=m_idx, column=3, value=col_stats['mismatch_count'])
                ws2.cell(row=m_idx, column=4, value=col_stats['total_comparisons'])
                ws2.cell(row=m_idx, column=5, value=f"{col_stats['mismatch_percentage']:.2f}%")
        else:
            # Если нет расхождений - выводим сообщение
            start_row = len(summary_data) + 2
            ws2.cell(row=start_row, column=1, value="Детали по колонкам с расхождениями").font = Font(bold=True)
            ws2.cell(row=start_row + 1, column=1, value="Расхождений в колонках не обнаружено")
        
        # Настраиваем ширину колонок
        for column in ws2.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = min(max_length + 2, 30)
            ws2.column_dimensions[column_letter].width = adjusted_width
        
        # Сохраняем файл
        output_file = output_path / f"comparison_report_{folder_name}.xlsx"
        wb.save(output_file)
        
        print(f"Отчет сохранен: {output_file}")
        print(f"Всего строк в отчете: {total_rows}")
        print(f"Строк с расхождениями: {rows_with_mismatches}")
        print(f"Всего расхождений: {all_mismatches_count}")
        print(f"Колонок с расхождениями: {len(columns_with_mismatches)}")
        
        # Детальная отладка по колонкам
        print("\nДетальная статистика по колонкам:")
        for col_stats in column_mismatch_stats:
            if col_stats['mismatch_count'] > 0:
                print(f"  {col_stats['system_a_column']} -> {col_stats['system_b_column']}: {col_stats['mismatch_count']} расхождений")
        
        return {
            'folder_name': folder_name,
            'total_rows': total_rows,
            'rows_with_mismatches': rows_with_mismatches,
            'rows_without_mismatches': rows_without_mismatches,
            'total_columns_a': total_columns_a,
            'total_columns_b': total_columns_b,
            'matched_columns': matched_columns,
            'unmatched_columns_a': unmatched_columns_a,
            'unmatched_columns_b': unmatched_columns_b,
            'total_mismatches': all_mismatches_count,
            'match_percentage': overall_match_percentage,
            'left_only_rows': left_only,
            'right_only_rows': right_only,
            'both_sides_rows': both_sides,
            'columns_with_mismatches': len(columns_with_mismatches)
        }
def create_summary_report(all_summary_data, output_path):
    """Создает сводный отчет по всем папкам с полной статистикой"""
    
    if not all_summary_data:
        print("Нет данных для сводного отчета")
        return
    
    # Создаем DataFrame с полной статистикой
    summary_records = []
    for stats in all_summary_data:
        summary_records.append({
            'Папка': stats['folder_name'],
            'Всего строк': stats['total_rows'],
            'Строк с расхождениями': stats['rows_with_mismatches'],
            'Строк без расхождений': stats['rows_without_mismatches'],
            '% строк с расхождениями': f"{(stats['rows_with_mismatches']/stats['total_rows'])*100:.2f}%" if stats['total_rows'] > 0 else "0%",
            'Колонок система A': stats['total_columns_a'],
            'Колонок система B': stats['total_columns_b'],
            'Сопоставленных колонок': stats['matched_columns'],
            'Всего расхождений': stats['total_mismatches'],
            '% совпадения': f"{stats['match_percentage']:.2f}%",
            'Только в системе A': stats.get('left_only_rows', 0),
            'Только в системе B': stats.get('right_only_rows', 0),
            'В обеих системах': stats.get('both_sides_rows', 0)
        })
    
    summary_df = pd.DataFrame(summary_records)
    
    wb = Workbook()
    ws = wb.active
    ws.title = "Сводный отчет по всем папкам"
    
    # Записываем заголовки
    headers = list(summary_df.columns)
    for col_idx, header in enumerate(headers, 1):
        cell = ws.cell(row=1, column=col_idx, value=header)
        cell.fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
        cell.font = Font(bold=True, color="FFFFFF")
        cell.alignment = Alignment(horizontal='center')
    
    # Записываем данные
    for row_idx, (_, row) in enumerate(summary_df.iterrows(), 2):
        for col_idx, value in enumerate(row, 1):
            ws.cell(row=row_idx, column=col_idx, value=value)
    
    # Добавляем итоги
    total_row = len(summary_df) + 2
    ws.cell(row=total_row, column=1, value="ИТОГО:").font = Font(bold=True)
    
    # Вычисляем итоги
    if not summary_df.empty:
        ws.cell(row=total_row, column=2, value=summary_df['Всего строк'].sum())
        ws.cell(row=total_row, column=3, value=summary_df['Строк с расхождениями'].sum())
        ws.cell(row=total_row, column=4, value=summary_df['Строк без расхождений'].sum())
        ws.cell(row=total_row, column=5, value=f"{(summary_df['Строк с расхождениями'].sum()/summary_df['Всего строк'].sum())*100:.2f}%")
        ws.cell(row=total_row, column=6, value=summary_df['Колонок система A'].sum())
        ws.cell(row=total_row, column=7, value=summary_df['Колонок система B'].sum())
        ws.cell(row=total_row, column=8, value=summary_df['Сопоставленных колонок'].sum())
        ws.cell(row=total_row, column=9, value=summary_df['Всего расхождений'].sum())
        
        # Средний процент совпадения
        avg_match = summary_df['% совпадения'].str.rstrip('%').astype(float).mean()
        ws.cell(row=total_row, column=10, value=f"{avg_match:.2f}%")
        
        ws.cell(row=total_row, column=11, value=summary_df['Только в системе A'].sum())
        ws.cell(row=total_row, column=12, value=summary_df['Только в системе B'].sum())
        ws.cell(row=total_row, column=13, value=summary_df['В обеих системах'].sum())
    
    # Настраиваем ширину колонок
    for column in ws.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 20)
        ws.column_dimensions[column_letter].width = adjusted_width
    
    output_file = output_path / "summary_report_all_folders.xlsx"
    wb.save(output_file)
    print(f"\nСводный отчет сохранен: {output_file}")
    
    # Выводим сводную статистику в консоль
    print("\n=== СВОДНАЯ СТАТИСТИКА ПО ВСЕМ ПАПКАМ ===")
    for stats in all_summary_data:
        print(f"\nПапка: {stats['folder_name']}")
        print(f"  Строк с расхождениями: {stats['rows_with_mismatches']}/{stats['total_rows']} ({stats['rows_with_mismatches']/stats['total_rows']*100:.1f}%)")
        print(f"  Всего расхождений: {stats['total_mismatches']}")
        print(f"  Процент совпадения: {stats['match_percentage']:.1f}%")
        
def process_all_folders_with_reports_fixed(base_folder_path, output_base_path):
    """Обрабатывает все папки с исправленной статистикой"""
    
    folder_pairs = analyze_folder_structure(base_folder_path)
    
    if not folder_pairs:
        print("Не найдено пар файлов для сравнения!")
        return
    
    output_path = Path(output_base_path)
    output_path.mkdir(exist_ok=True)
    
    comparator = AdvancedColumnComparator(similarity_threshold=75)
    all_summary_data = []
    
    for pair in folder_pairs:
        print(f"\n{'='*50}")
        print(f"Обработка папки: {pair['folder']}")
        print(f"{'='*50}")
        
        try:
            # Чтение файлов
            df1 = pd.read_excel(pair['file1'])
            df2 = pd.read_excel(pair['file2'])
            
            print(f"Система A: {len(df1)} строк, {len(df1.columns)} колонок")
            print(f"Система B: {len(df2)} строк, {len(df2.columns)} колонок")
            
            # Поиск соответствий
            matches_df, profiles1, profiles2 = comparator.find_column_matches(df1, df2)
            
            # Автоматически находим ключевые колонки
            key_col_a, key_col_b = find_best_key_columns(df1, df2, matches_df)
            
            if key_col_a and key_col_b:
                print(f"Ключевые колонки: {key_col_a} -> {key_col_b}")
                
                # Создаем детальный отчет
                summary_stats = comparator.create_detailed_mismatch_report(
                    df1, df2, matches_df, (key_col_a, key_col_b), 
                    pair['folder'], output_path
                )
                
                all_summary_data.append(summary_stats)
                
                # Выводим краткую статистику
                matched_count = len(matches_df[matches_df['status'] == 'MATCHED'])
                print(f"Сопоставлено колонок: {matched_count}")
                print(f"Строк с расхождениями: {summary_stats['rows_with_mismatches']}")
                print(f"Всего расхождений: {summary_stats['total_mismatches']}")
                print(f"Общий процент совпадения: {summary_stats['match_percentage']:.2f}%")
            else:
                print("Не удалось найти подходящие ключевые колонки для сравнения")
                
        except Exception as e:
            print(f"Ошибка при обработке {pair['folder']}: {str(e)}")
            import traceback
            traceback.print_exc()
    
    # Создаем сводный отчет по всем папкам
    if all_summary_data:
        create_summary_report(all_summary_data, output_path)
    
    return all_summary_data

In [40]:

def debug_data_loading(file_path):
    """Функция для отладки загрузки данных"""
    print(f"\n=== ОТЛАДКА ЗАГРУЗКИ ФАЙЛА: {file_path} ===")
    
    try:
        # Пробуем прочитать файл
        df = pd.read_excel(file_path)
        print(f"Успешно загружено: {len(df)} строк, {len(df.columns)} колонок")
        print("Колонки:", list(df.columns))
        print("Первые 3 строки:")
        print(df.head(3))
        print("\nТипы данных:")
        print(df.dtypes)
        return df
    except Exception as e:
        print(f"Ошибка загрузки: {e}")
        return None

def process_all_folders_with_reports_debug(base_folder_path, output_base_path):
    """Обрабатывает все папки с дополнительной отладкой"""
    
    folder_pairs = analyze_folder_structure(base_folder_path)
    
    if not folder_pairs:
        print("Не найдено пар файлов для сравнения!")
        return
    
    output_path = Path(output_base_path)
    output_path.mkdir(exist_ok=True)
    
    comparator = AdvancedColumnComparator(similarity_threshold=75)
    all_summary_data = []
    
    for pair in folder_pairs:
        print(f"\n{'='*50}")
        print(f"Обработка папки: {pair['folder']}")
        print(f"{'='*50}")
        
        try:
            # Загружаем файлы с отладкой
            print("Загрузка файла 1...")
            df1 = debug_data_loading(pair['file1'])
            print("\nЗагрузка файла 2...")
            df2 = debug_data_loading(pair['file2'])
            
            if df1 is None or df2 is None:
                print("Пропускаем папку из-за ошибок загрузки")
                continue
            
            # Поиск соответствий
            print("\nПоиск соответствий между колонками...")
            matches_df, profiles1, profiles2 = comparator.find_column_matches(df1, df2)
            print("Результаты сопоставления:")
            print(matches_df)
            
            # Автоматически находим ключевые колонки
            key_col_a, key_col_b = find_best_key_columns(df1, df2, matches_df)
            
            if key_col_a and key_col_b:
                print(f"Выбраны ключевые колонки: {key_col_a} -> {key_col_b}")
                
                # Создаем детальный отчет
                summary_stats = comparator.create_detailed_mismatch_report(
                    df1, df2, matches_df, (key_col_a, key_col_b), 
                    pair['folder'], output_path
                )
                
                all_summary_data.append(summary_stats)
                
                # Выводим краткую статистику
                matched_count = len(matches_df[matches_df['status'] == 'MATCHED'])
                print(f"Сопоставлено колонок: {matched_count}")
                print(f"Строк с расхождениями: {summary_stats['rows_with_mismatches']}")
                print(f"Общий процент совпадения: {summary_stats['match_percentage']:.2f}%")
            else:
                print("Не удалось найти подходящие ключевые колонки для сравнения")
                
        except Exception as e:
            print(f"Ошибка при обработке {pair['folder']}: {str(e)}")
            import traceback
            traceback.print_exc()
    
    # Создаем сводный отчет по всем папкам
    if all_summary_data:
        create_summary_report(all_summary_data, output_path)
    
    return all_summary_data

# ДЕМОНСТРАЦИЯ С РЕАЛЬНЫМИ ДАННЫМИ
def demo_with_real_data():
    """Демонстрация с реалистичными тестовыми данными"""
    print("=== ТЕСТИРОВАНИЕ С РЕАЛЬНЫМИ ДАННЫМИ ===\n")
    
    # Создаем тестовые данные
    data_a = {
        'ID': [1, 2, 3, 4, 5],
        'Product_Name': ['Apple iPhone', 'Samsung Galaxy', 'Google Pixel', 'Xiaomi Mi', 'Huawei P'],
        'Price': [999.99, 899.99, 799.99, 499.99, 599.99],
        'Quantity': [100, 150, 80, 200, 120],
        'Category': ['Smartphone', 'Smartphone', 'Smartphone', 'Smartphone', 'Smartphone']
    }
    
    data_b = {
        'ProductID': [1, 2, 3, 4, 6],  # ID 5 отсутствует, добавлен ID 6
        'Name': ['Apple iPhone', 'Samsung Galaxy S', 'Google Pixel', 'Xiaomi Mi', 'OnePlus'],
        'Cost': [999.99, 949.99, 799.99, 499.99, 699.99],  # Изменена цена Samsung
        'Stock': [100, 150, 75, 200, 90],  # Изменен stock Google
        'Type': ['Phone', 'Phone', 'Phone', 'Phone', 'Phone']  # Разные названия категорий
    }
    
    df_a = pd.DataFrame(data_a)
    df_b = pd.DataFrame(data_b)
    
    print("Система A:")
    print(df_a)
    print("\nСистема B:")
    print(df_b)
    
    # Сохраняем в временные файлы
    temp_path = Path("./temp_test")
    temp_path.mkdir(exist_ok=True)
    
    df_a.to_excel(temp_path / "system_a.xlsx", index=False)
    df_b.to_excel(temp_path / "system_b.xlsx", index=False)
    
    print(f"\nФайлы сохранены в: {temp_path}")
    
    # Тестируем загрузку
    print("\nТестируем загрузку файлов...")
    loaded_a = debug_data_loading(temp_path / "system_a.xlsx")
    loaded_b = debug_data_loading(temp_path / "system_b.xlsx")
    
    if loaded_a is not None and loaded_b is not None:
        # Запускаем сравнение
        comparator = AdvancedColumnComparator()
        matches_df, _, _ = comparator.find_column_matches(loaded_a, loaded_b)
        
        print("\nРезультаты сопоставления:")
        print(matches_df)
        
        # Создаем отчет
        summary_stats = comparator.create_detailed_mismatch_report(
            loaded_a, loaded_b, matches_df, ('ID', 'ProductID'), 
            'test_folder', temp_path
        )
        
        print(f"\nСтатистика отчета:")
        for key, value in summary_stats.items():
            print(f"  {key}: {value}")
    
    # Очистка
    import shutil
    shutil.rmtree(temp_path)
    
    return comparator, matches_df, summary_stats


In [41]:

def process_all_folders_with_reports(base_folder_path, output_base_path):
    """Обрабатывает все папки и создает отчеты"""
    
    folder_pairs = analyze_folder_structure(base_folder_path)
    
    if not folder_pairs:
        print("Не найдено пар файлов для сравнения!")
        return
    
    output_path = Path(output_base_path)
    output_path.mkdir(exist_ok=True)
    
    comparator = AdvancedColumnComparator(similarity_threshold=75)
    all_summary_data = []
    
    for pair in folder_pairs:
        print(f"\n{'='*50}")
        print(f"Обработка папки: {pair['folder']}")
        print(f"{'='*50}")
        
        try:
            # Чтение файлов
            df1 = pd.read_excel(pair['file1'])
            df2 = pd.read_excel(pair['file2'])
            
            print(f"Система A: {len(df1)} строк, {len(df1.columns)} колонок")
            print(f"Система B: {len(df2)} строк, {len(df2.columns)} колонок")
            
            # Поиск соответствий
            matches_df, profiles1, profiles2 = comparator.find_column_matches(df1, df2)
            
            # Автоматически находим ключевые колонки
            key_col_a, key_col_b = find_best_key_columns(df1, df2, matches_df)
            
            if key_col_a and key_col_b:
                print(f"Ключевые колонки: {key_col_a} -> {key_col_b}")
                
                # Создаем детальный отчет
                summary_stats = comparator.create_detailed_mismatch_report(
                    df1, df2, matches_df, (key_col_a, key_col_b), 
                    pair['folder'], output_path
                )
                
                all_summary_data.append(summary_stats)
                
                # Выводим краткую статистику
                matched_count = len(matches_df[matches_df['status'] == 'MATCHED'])
                print(f"Сопоставлено колонок: {matched_count}")
                print(f"Строк с расхождениями: {summary_stats['rows_with_mismatches']}")
                print(f"Общий процент совпадения: {summary_stats['match_percentage']:.2f}%")
            else:
                print("Не удалось найти подходящие ключевые колонки для сравнения")
                
        except Exception as e:
            print(f"Ошибка при обработке {pair['folder']}: {str(e)}")
            import traceback
            traceback.print_exc()
    
    # Создаем сводный отчет по всем папкам
    if all_summary_data:
        create_summary_report(all_summary_data, output_path)
    
    return all_summary_data

def find_best_key_columns(df1, df2, matches_df):
    """Находит лучшие ключевые колонки для соединения"""
    
    # Ищем сопоставленные колонки с высокой уникальностью
    matched_pairs = matches_df[matches_df['status'] == 'MATCHED']
    
    key_candidates = []
    
    for _, match in matched_pairs.iterrows():
        col_a = match['system_a_column']
        col_b = match['system_b_column']
        
        if col_a in df1.columns and col_b in df2.columns:
            unique_pct_a = (df1[col_a].nunique() / len(df1)) * 100
            unique_pct_b = (df2[col_b].nunique() / len(df2)) * 100
            
            # Предпочтение колонкам с высокой уникальностью и хорошим score
            if unique_pct_a > 90 and unique_pct_b > 90:
                score = (unique_pct_a + unique_pct_b + match['combined_score']) / 3
                key_candidates.append((col_a, col_b, score))
    
    if key_candidates:
        # Выбираем лучшую пару по комбинированному score
        best_candidate = max(key_candidates, key=lambda x: x[2])
        return best_candidate[0], best_candidate[1]
    
    # Если не нашли, пробуем найти по отдельности
    key_col_a = find_best_single_key_column(df1)
    key_col_b = find_best_single_key_column(df2)
    
    return key_col_a, key_col_b

def find_best_single_key_column(df):
    """Находит лучшую ключевую колонку в одном DataFrame"""
    best_col = None
    best_score = 0
    
    for col in df.columns:
        unique_pct = (df[col].nunique() / len(df)) * 100
        null_pct = (df[col].isnull().sum() / len(df)) * 100
        
        # Score учитывает уникальность и отсутствие null
        score = unique_pct * (100 - null_pct) / 100
        
        if score > best_score and unique_pct > 80 and null_pct < 5:
            best_score = score
            best_col = col
    
    return best_col

def create_summary_report(all_summary_data, output_path):
    """Создает сводный отчет по всем папкам"""
    
    summary_df = pd.DataFrame(all_summary_data)
    
    wb = Workbook()
    ws = wb.active
    ws.title = "Сводный отчет по всем папкам"
    
    # Записываем данные
    for r_idx, row in enumerate(dataframe_to_rows(summary_df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            cell = ws.cell(row=r_idx, column=c_idx, value=value)
            if r_idx == 1:
                cell.fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
                cell.font = Font(bold=True, color="FFFFFF")
                cell.alignment = Alignment(horizontal='center')
    
    # Добавляем итоги
    if not summary_df.empty:
        total_row = len(summary_df) + 2
        ws.cell(row=total_row, column=1, value="ИТОГО:").font = Font(bold=True)
        
        ws.cell(row=total_row, column=2, value=summary_df['total_rows'].sum())
        ws.cell(row=total_row, column=3, value=summary_df['rows_with_mismatches'].sum())
        ws.cell(row=total_row, column=4, value=summary_df['rows_without_mismatches'].sum())
        ws.cell(row=total_row, column=5, value=summary_df['total_columns_a'].sum())
        ws.cell(row=total_row, column=6, value=summary_df['total_columns_b'].sum())
        ws.cell(row=total_row, column=7, value=summary_df['matched_columns'].sum())
        ws.cell(row=total_row, column=8, value=f"{summary_df['match_percentage'].mean():.2f}%")
    
    # Настраиваем ширину колонок
    for column in ws.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 20)
        ws.column_dimensions[column_letter].width = adjusted_width
    
    output_file = output_path / "summary_report_all_folders.xlsx"
    wb.save(output_file)
    print(f"\nСводный отчет сохранен: {output_file}")

# ДЕМОНСТРАЦИЯ РАБОТЫ
def demo_advanced_functionality():
    """Демонстрация расширенного функционала"""
    print("=== ДЕМОНСТРАЦИЯ РАСШИРЕННОГО ФУНКЦИОНАЛА ===\n")
    
    # Создаем тестовые данные с преднамеренными расхождениями
    data_a = {
        'ID': ['A001', 'A002', 'A003', 'A004', 'A005'],
        'Name': ['Product 1', 'Product 2', 'Product 3', 'Product 4', 'Product 5'],
        'Price': [100.50, 200.75, 150.00, 300.25, 250.50],
        'Quantity': [10, 25, 30, 5, 15],
        'Category': ['CAT1', 'CAT2', 'CAT1', 'CAT3', 'CAT2']
    }
    
    data_b = {
        'ProductID': ['A001', 'A002', 'A003', 'A004', 'A006'],  # A005 отсутствует, A006 новый
        'ProductName': ['Product 1', 'Product 2 Changed', 'Product 3', 'Product 4', 'Product 6'],
        'Cost': [100.50, 250.75, 150.00, 300.25, 350.00],  # Price -> Cost, одно значение изменено
        'Qty': [10, 25, 35, 5, 20],  # Quantity -> Qty, одно значение изменено
        'Type': ['CAT1', 'CAT2', 'CAT1', 'CAT3', 'CAT4']  # Category -> Type
    }
    
    df_a = pd.DataFrame(data_a)
    df_b = pd.DataFrame(data_b)
    
    print("Демонстрационные данные созданы с преднамеренными расхождениями")
    print("Система A:")
    print(df_a)
    print("\nСистема B:")
    print(df_b)
    
    # Создаем временную папку для демо
    temp_path = Path("./temp_demo")
    temp_path.mkdir(exist_ok=True)
    
    # Запускаем расширенный анализ
    comparator = AdvancedColumnComparator()
    matches_df, _, _ = comparator.find_column_matches(df_a, df_b)
    
    print(f"\nНайдено сопоставлений: {len(matches_df[matches_df['status'] == 'MATCHED'])}")
    
    # Создаем детальный отчет
    summary_stats = comparator.create_detailed_mismatch_report(
        df_a, df_b, matches_df, ('ID', 'ProductID'), 
        'demo_folder', temp_path
    )
    
    print(f"\nСтатистика отчета:")
    for key, value in summary_stats.items():
        print(f"  {key}: {value}")
    
    # Очистка
    import shutil
    shutil.rmtree(temp_path)
    
    return comparator, matches_df, summary_stats

# Запуск демо
# comparator, matches, stats = demo_advanced_functionality()

In [42]:
def process_real_files(base_folder_path):
    """Обработка реальных Excel файлов из папок"""
    
    # 1. Анализ структуры папок
    folder_pairs = analyze_folder_structure(base_folder_path)
    
    if not folder_pairs:
        print("Не найдено пар файлов для сравнения!")
        return
    
    print(f"Найдено пар для сравнения: {len(folder_pairs)}")
    
    # 2. Инициализация компаратора
    comparator = ColumnComparator(similarity_threshold=75)
    
    all_results = {}
    
    for pair in folder_pairs:
        print(f"\n{'='*50}")
        print(f"Обработка папки: {pair['folder']}")
        print(f"Файл 1: {pair['file1'].name}")
        print(f"Файл 2: {pair['file2'].name}")
        print(f"{'='*50}")
        
        try:
            # Чтение файлов
            df1 = pd.read_excel(pair['file1'])
            df2 = pd.read_excel(pair['file2'])
            
            print(f"Система A: {len(df1)} строк, {len(df1.columns)} колонок")
            print(f"Система B: {len(df2)} строк, {len(df2.columns)} колонок")
            
            # Поиск соответствий
            matches_df, profiles1, profiles2 = comparator.find_column_matches(df1, df2)
            
            # Сохраняем результаты
            all_results[pair['folder']] = {
                'matches': matches_df,
                'profiles_a': profiles1,
                'profiles_b': profiles2,
                'dataframes': (df1, df2)
            }
            
            # Выводим результаты
            matched_count = len(matches_df[matches_df['status'] == 'MATCHED'])
            print(f"\nНайдено соответствий: {matched_count}/{len(df1.columns)}")
            
            # Показываем лучшие соответствия
            best_matches = matches_df[matches_df['status'] == 'MATCHED'].nlargest(5, 'combined_score')
            if not best_matches.empty:
                print("\nЛучшие соответствия:")
                for _, match in best_matches.iterrows():
                    print(f"  {match['system_a_column']} -> {match['system_b_column']} "
                          f"(score: {match['combined_score']:.1f})")
            
            # Показываем несопоставленные колонки
            unmatched = matches_df[matches_df['status'] == 'UNMATCHED']
            if not unmatched.empty:
                print(f"\nНесопоставленные колонки ({len(unmatched)}):")
                for col in unmatched['system_a_column'].head(3):
                    print(f"  {col}")
                if len(unmatched) > 3:
                    print(f"  ... и еще {len(unmatched) - 3}")
                    
        except Exception as e:
            print(f"Ошибка при обработке {pair['folder']}: {str(e)}")
    
    return all_results

# Функция для детального анализа конкретной пары
def detailed_analysis(results, folder_name):
    """Детальный анализ для конкретной пары файлов"""
    if folder_name not in results:
        print("Папка не найдена в результатах!")
        return
    
    data = results[folder_name]
    matches_df = data['matches']
    df1, df2 = data['dataframes']
    
    print(f"Детальный анализ для: {folder_name}")
    print("\nПолная таблица соответствий:")
    display(matches_df.sort_values('combined_score', ascending=False))
    
    # Сравнение значений
    print("\nСравнение значений в сопоставленных колонках:")
    
    # Автоматически находим ключевые колонки (самые уникальные)
    key_candidates_a = []
    key_candidates_b = []
    
    for col in df1.columns:
        unique_pct = (df1[col].nunique() / len(df1)) * 100
        if unique_pct > 95:  # Очень высокая уникальность
            key_candidates_a.append((col, unique_pct))
    
    for col in df2.columns:
        unique_pct = (df2[col].nunique() / len(df2)) * 100
        if unique_pct > 95:
            key_candidates_b.append((col, unique_pct))
    
    if key_candidates_a and key_candidates_b:
        key_col_a = max(key_candidates_a, key=lambda x: x[1])[0]
        key_col_b = max(key_candidates_b, key=lambda x: x[1])[0]
        print(f"Автовыбранные ключевые колонки: {key_col_a} -> {key_col_b}")
        
        comparator = ColumnComparator()
        comparison_results = comparator.compare_matched_columns(
            df1, df2, matches_df, 
            key_columns=(key_col_a, key_col_b)
        )
        display(comparison_results)
    else:
        print("Не удалось автоматически найти подходящие ключевые колонки")

In [15]:
   # Для работы с реальными файлами
base_folder = "comparison_data"  # Укажите ваш путь
all_results = process_real_files(base_folder)

Найдено пар для сравнения: 1

Обработка папки: пара1
Файл 1: TMC_clean.xlsx
Файл 2: TMC_clean2.xlsx
Система A: 104567 строк, 3 колонок
Система B: 104567 строк, 3 колонок

Найдено соответствий: 3/3

Лучшие соответствия:
  CSCD_ID -> CSCD_ID/код (score: 91.2)
  FULL_NAME/ru_RU -> FULL_NAME/ru_RU/Полное имя (score: 89.2)
  Hierarchy_MTR_Class -> Hierarchy (score: 85.6)


In [16]:
   # Для детального анализа конкретной пары
if all_results:
    detailed_analysis(all_results, 'пара1')  # Укажите имя папки

Детальный анализ для: пара1

Полная таблица соответствий:


,system_a_column,system_b_column,name_similarity,data_similarity,combined_score,status
1,CSCD_ID,CSCD_ID/код,78,100.0,91.2,MATCHED
2,FULL_NAME/ru_RU,FULL_NAME/ru_RU/Полное имя,73,100.0,89.2,MATCHED
0,Hierarchy_MTR_Class,Hierarchy,64,100.0,85.6,MATCHED



Сравнение значений в сопоставленных колонках:
Автовыбранные ключевые колонки: CSCD_ID -> CSCD_ID/код


""


In [45]:
# Полная обработка всех папок
base_path = "comparison_data"
output_path = "comparison_data"
results = process_all_folders_with_reports_fixed(base_path, output_path)




Обработка папки: пара1
Система A: 104567 строк, 3 колонок
Система B: 104567 строк, 3 колонок
Ключевые колонки: CSCD_ID -> CSCD_ID/код
Начинаем создание отчета...
df1 колонки: ['Hierarchy_MTR_Class', 'CSCD_ID', 'FULL_NAME/ru_RU']
df2 колонки: ['Hierarchy', 'CSCD_ID/код', 'FULL_NAME/ru_RU/Полное имя']
Key columns: ('CSCD_ID', 'CSCD_ID/код')
Сопоставленные пары: 3
После merge: 104568 строк
Колонки после merge: ['Hierarchy_MTR_Class', 'CSCD_ID', 'FULL_NAME/ru_RU', 'Hierarchy', 'CSCD_ID/код', 'FULL_NAME/ru_RU/Полное имя', '_merge']
Отчет сохранен: comparison_data\comparison_report_пара1.xlsx
Всего строк в отчете: 104568
Строк с расхождениями: 9
Всего расхождений: 13
Колонок с расхождениями: 3

Детальная статистика по колонкам:
  Hierarchy_MTR_Class -> Hierarchy: 4 расхождений
  CSCD_ID -> CSCD_ID/код: 2 расхождений
  FULL_NAME/ru_RU -> FULL_NAME/ru_RU/Полное имя: 7 расхождений
Сопоставлено колонок: 3
Строк с расхождениями: 9
Всего расхождений: 13
Общий процент совпадения: 100.00%

Сводный 